In [1]:
import wikipedia
import nltk
import re
from pymystem3 import Mystem
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/a.tsigankov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
wikipedia.set_lang('ru')
tokenizer = nltk.RegexpTokenizer(r"\w+")
mystem = Mystem()

Факты

In [3]:
facts = [
    "Нацистский врач-эсэсовец, работавший в шести концлагерях, был дважды оправдан и выпущен на свободу.",
    "Историк утверждает, что, прежде чем допустить крестьян на личный прием к вождю мирового пролетариата, их тщательно дезинфицировали.",
    "В России акции протеста проходят не только на площадях, но и на поездах."
]

Название статей Википедии, на которые ссылаются факты

In [4]:
wikipedia_pages = {
    1: ["Шмидт, Генрих (врач)"],
    2: ["Ходоки у В. И. Ленина", "Дезинфекция"],
    3: ["Проезд снаружи поездов"]
}

### Скачивание и обработка статей из Википедии

In [5]:
def get_page_text(pagename):
    page = wikipedia.page(pagename)
    return page.content

In [6]:
def remove_trash(page_text):
    page_text = re.sub('\s', ' ', page_text)
    page_text = re.sub('[А-Я]\.', '', page_text)
    page_text = re.sub(r'\=\= Примечания[\w\s\=]*', '', page_text)
    page_text = page_text.replace('.', '. ')
    page_text = page_text.replace('\n', ' ')
    headers = re.findall(r"\=\=.*?\=\=", page_text)
    for header in headers:
        page_text = page_text.replace(header, '')
    
    return page_text

In [26]:
documents = []

for num in wikipedia_pages.keys():
    for page in wikipedia_pages[num]:
        page_text = remove_trash(get_page_text(page))
        sentences = nltk.sent_tokenize(page_text, language="russian")
        documents.extend(sentences)

In [28]:
documents.extend(facts)

In [29]:
wiki = pd.DataFrame(documents)

In [30]:
wiki.columns = ['document']

In [31]:
wiki.head()

,document
0,Эрнст Генрих Шмидт (нем. Ernst Heinrich Schmi...
1,Генрих Шмидт родился 27 марта 1912 года в Альт...
2,В 1933 году был зачислен в СС (№ 23 069).
3,В 1937 году получил степень доктора медицински...
4,После начала Второй мировой войны работал в ла...


### Лемматизация 

In [32]:
def lemmatize(sentence):
    return ''.join((mystem.lemmatize(' '.join(tokenizer.tokenize(sentence))))).replace('\n', '')

In [33]:
wiki['lemmatized_document'] = wiki.document.apply(lemmatize)

In [34]:
wiki.tail()

,document,lemmatized_document
561,Дата обращения 7 декабря 2012.,дата обращение 7 декабрь 2012
562,Архивировано 27 декабря 2012 года.,архивировать 27 декабрь 2012 год
563,"Нацистский врач-эсэсовец, работавший в шести к...",нацистский врач эсэсовец работать в шесть конц...
564,"Историк утверждает, что, прежде чем допустить ...",историк утверждать что прежде чем допускать кр...
565,В России акции протеста проходят не только на ...,в россия акция протест проходить не только на ...


In [35]:
tfidf_vect = TfidfVectorizer()
tfidf_weight = tfidf_vect.fit_transform(wiki['lemmatized_document'])

In [39]:
nn_cosine = NearestNeighbors(metric='cosine')
nn_cosine.fit(tfidf_weight)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [49]:
# facts indexes
# 563
# 564
# 565

In [55]:
def get_nearest_documents(index):
    cosine, indices = nn_cosine.kneighbors(tfidf_weight[index], n_neighbors = 11)

    neighbors_cosine = pd.DataFrame({'cosine': cosine.flatten(), 'id': indices.flatten()})

    nearest_documents = (wiki.\
                    merge(neighbors_cosine, right_on = 'id', left_index = True).\
                    sort_values('cosine')[['id', 'document', 'cosine']])

    return nearest_documents

In [56]:
get_nearest_documents(563)

,id,document,cosine
0,563,"Нацистский врач-эсэсовец, работавший в шести к...",0.000000
1,10,В итоге он был оправдан.,0.764327
2,4,После начала Второй мировой войны работал в ла...,0.765391
3,5,В октябре 1943 года стал главным врачом в конц...,0.819377
4,216,"Предметы, жесты и позы персонажей работают на ...",0.892995
5,123,В 1950 году на Ленинградском фарфоровом заводе...,0.895028
6,254,С некоторых пор и в России они получили распро...,0.896276
7,0,Эрнст Генрих Шмидт (нем. Ernst Heinrich Schmi...,0.903971
8,13,Из-за отсутствия доказательств 20 марта 1979 г...,0.905513
9,39,"Мария Скрипник, работавшая в Совнаркоме по при...",0.913396
